## SCRAPPING MAIIA

**Contexte du code** 

Ce code a pour objectif de récupérer les URLs de chaque médecin généraliste référencé sur le site Maiia dans une région spécifiée. Une fois les URLs collectées, il parcourt chacune des pages correspondantes pour extraire des informations prédéfinies à partir du code source, puis les ajoute à un tableau de données.


### CREATION D'UN PREMIER DATAFRAME

Je creer un dataframe qui regroupe 
- l'URL
- ADRESSE 
- CONVENTION

a partir de l'url et d'un petit traitement de donnée je peux récup 
- ID
- Medical specialty
- consultation availability etc..


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random

# Liste de User-Agent pour changer à chaque requête
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59"
]


### modifier le chemin d'accées en fonction de la regions souhaité
url = "https://www.maiia.com/medecin-generaliste/11-ILE-DE-FRANCE?page="
medecin_urls = []

for i in range(1, 5):
    try:
        # Choisir un User-Agent aléatoire
        headers = {
            "User-Agent": random.choice(user_agents)
        }

        # Faire la requête de la page HTLM
        page = requests.get(url + str(i), headers=headers)
        page.raise_for_status()  # Vérifier si la requête a échoué
        soup = BeautifulSoup(page.text, "html.parser")

        # Sélectionner tous les liens des médecins
        medecin_lien = soup.select("a.mui-1wisq94")

        # Extraire les URLs valides
        urls_page = ["https://www.maiia.com" + lien["href"] for lien in medecin_lien if lien.has_attr("href")]
        
        # Ajouter les URLs extraites à la liste
        medecin_urls.extend(urls_page)

        # Attendre un délai aléatoire entre chaque requête pour ne pas surcharger les serveur du site 
        time.sleep(random.uniform(2, 15))

    except requests.exceptions.RequestException as e:
        print(f"Erreur avec la page {i}: {e}")

# Afficher les résultats
#print(medecin_urls)

In [16]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

# Liste de User-Agent pour changer à chaque requête
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59"
]

# URL de base pour
url = "https://www.maiia.com/medecin-generaliste/11-ILE-DE-FRANCE?page="


medecin_urls = []
adresses = []
conventions = []


for i in range(1, 2):
    try:
        # Choisir un User-Agent aléatoire
        headers = {
            "User-Agent": random.choice(user_agents)
        }

        # Faire la requête HTTP et récupérer le contenu de la page
        page = requests.get(url + str(i), headers=headers)
        page.raise_for_status()  
        soup = BeautifulSoup(page.text, "html.parser")

        # Sélection des liens des médecins
        medecin_lien = soup.select("a.mui-1wisq94")

        # Sélection des informations supplémentaires
        adresses_med = soup.select(".prat__address")
        convention_med = soup.select(".prat__secu")

        # Extraire les URLs des médecins
        urls_page = ["https://www.maiia.com" + lien["href"] for lien in medecin_lien if lien.has_attr("href")]
        medecin_urls.extend(urls_page)

        # Extraire les adresses
        adresses_page = [adresse.get_text(strip=True) for adresse in adresses_med]
        adresses.extend(adresses_page)

        # Extraire les informations de convention
        conventions_page = [convention.get_text(strip=True) for convention in convention_med]
        conventions.extend(conventions_page)

        # delais
        time.sleep(random.uniform(2, 16))

    except requests.exceptions.RequestException as e:
        print(f"Erreur avec la page {i}: {e}")
        continue  # Passer à la page suivante si une erreur se produit

# Créer un DataFrame 
data = {
    "URL": medecin_urls,
    "Adresse": adresses,
    "Convention": conventions
}

df = pd.DataFrame(data)

df.to_csv("base_BS_ILE_DE_FRANCE.csv", index=False, encoding="utf-8")


### SELENIUM SCRAPING DES PAGES MEDECINS

In [23]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

# Liste de User-Agent pour changer à chaque requête
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59"
]

# URL de base pour
url = "https://www.maiia.com/medecin-generaliste/11-ILE-DE-FRANCE?page="


medecin_urls = []
adresses = []
conventions = []


for i in range(1, 70):
    try:
        # Choisir un User-Agent aléatoire
        headers = {
            "User-Agent": random.choice(user_agents)
        }

        # Faire la requête HTTP et récupérer le contenu de la page
        page = requests.get(url + str(i), headers=headers)
        page.raise_for_status()  
        soup = BeautifulSoup(page.text, "html.parser")

        # Sélection des liens des médecins
        medecin_lien = soup.select("a.mui-1wisq94")

        # Sélection des informations supplémentaires
        adresses_med = soup.select(".prat__address")
        convention_med = soup.select(".prat__secu")

        # Extraire les URLs des médecins
        urls_page = ["https://www.maiia.com" + lien["href"] for lien in medecin_lien if lien.has_attr("href")]
        medecin_urls.extend(urls_page)

        # Extraire les adresses
        adresses_page = [adresse.get_text(strip=True) for adresse in adresses_med]
        adresses.extend(adresses_page)

        # Extraire les informations de convention
        conventions_page = [convention.get_text(strip=True) for convention in convention_med]
        conventions.extend(conventions_page)

        # delais
        time.sleep(random.uniform(2, 16))

    except requests.exceptions.RequestException as e:
        print(f"Erreur avec la page {i}: {e}")
        continue  # Passer à la page suivante si une erreur se produit

# Créer un DataFrame 
data = {
    "URL": medecin_urls,
    "Adresse": adresses,
    "Conventionbs": conventions
}

df = pd.DataFrame(data)

df.to_csv("BASE_SECOURS/BS_ILE_DE_FRANCE.csv",sep=';', index=False, encoding="utf-8")


Erreur avec la page 31: HTTPSConnectionPool(host='www.maiia.com', port=443): Max retries exceeded with url: /medecin-generaliste/11-ILE-DE-FRANCE?page=31 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14103c5d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Erreur avec la page 32: HTTPSConnectionPool(host='www.maiia.com', port=443): Max retries exceeded with url: /medecin-generaliste/11-ILE-DE-FRANCE?page=32 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14103c250>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Erreur avec la page 33: HTTPSConnectionPool(host='www.maiia.com', port=443): Max retries exceeded with url: /medecin-generaliste/11-ILE-DE-FRANCE?page=33 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1414b6d50>: Failed to establish a new connection: [Errno 8] nodename nor servname pro

In [ ]:

import logging
import re
import time
import concurrent.futures
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configuration du navigateur
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

logging.basicConfig(filename='scraping_errors.log', level=logging.DEBUG)

def init_driver():
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scrape_url(url):
    driver = init_driver()
    result = {
        'URL': url,
        'Disponibilité': None,
        'Méthodes de paiement': None,
        'Tarifs': None,
        'Date de diplôme': None,
        'Horaires': None
    }
    
    try:

        # Accéder à la page du médecin
        driver.get(url)
        
        # Attendre que la page soit chargée
        WebDriverWait(driver, 20).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )

        # Récupérer la disponibilité

        # Récupérer la disponibilité en ligne
        try:
            # Chercher l'élément avec la classe 'info' qui indique qu'il n'y a plus de disponibilité
            info_elements = driver.find_elements(By.CSS_SELECTOR, "p.info")
            if info_elements:
                result['Disponibilité en ligne'] = info_elements[0].text.strip()
            else:
                # Si cet élément n'est pas trouvé, continuer normalement avec les autres étapes
                disponibilite_elements = WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.calendar-info-availability span.info-availability span.date.percy-hidden"))
                )
                if disponibilite_elements:
                    result['Disponibilité'] = disponibilite_elements[0].text.strip()
        except Exception as e:
            logging.warning(f"Disponibilité non trouvée pour {url}: {e}")

        # Tarifs
        try:
            tarif_med = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.MuiTableRow-root th.MuiTableCell-root + td.MuiTableCell-root"))
            )
            if tarif_med:
                tarifs = [element.text for element in tarif_med]
                result['Tarifs'] = ', '.join(tarifs)
        except Exception as e:
            logging.warning(f"Tarifs non trouvés pour {url}: {e}")

        # Méthodes de paiement - Solution améliorée
        try:
            # Attendre que la section soit présente (pas nécessairement visible)
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//h3[contains(., 'Méthodes de paiement')]")))
            
            # Faire défiler la page vers l'élément pour le rendre visible
            payment_title = driver.find_element(By.XPATH, "//h3[contains(., 'Méthodes de paiement')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", payment_title)
            time.sleep(1)  # Petit délai pour le défilement
            
            # Trouver la section des méthodes de paiement
            payment_section = driver.find_element(By.XPATH, "//h3[contains(., 'Méthodes de paiement')]/following-sibling::div")
            
            # Récupérer toutes les méthodes actives (sans la classe tag--ko)
            methods = []
            tags = payment_section.find_elements(By.CLASS_NAME, "tag")
            for tag in tags:
                if 'tag--ko' not in tag.get_attribute('class'):
                    methods.append(tag.text.strip())
            
            if methods:
                result['Méthodes de paiement'] = ', '.join(methods)
            else:
                logging.warning(f"Aucune méthode de paiement active trouvée pour {url}")
        except Exception as e:
            logging.error(f"Erreur méthodes de paiement pour {url}: {e}")
            # Essayez de récupérer le HTML pour déboguer
            try:
                with open(f"error_page_{result['ID']}.html", "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
            except:
                pass

        # Convention - Solution similaire à Méthodes de paiement
        try:
            # Attendre que la section des conventions soit présente
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'MuiBox-root') and contains(., 'Conventionné')]")))
            
            # Trouver la section des conventions
            convention_section = driver.find_element(By.XPATH, "//div[contains(@class, 'MuiBox-root') and contains(., 'Conventionné')]")
            
            # Récupérer toutes les conventions actives (sans la classe tag--ko)
            conventions = []
            convention_tags = convention_section.find_elements(By.CLASS_NAME, "tag")
            for tag in convention_tags:
                if 'tag--ko' not in tag.get_attribute('class'):
                    conventions.append(tag.text.strip())
            
            if conventions:
                result['Convention'] = ', '.join(conventions)
            else:
                logging.warning(f"Aucune convention active trouvée pour {url}")
        except Exception as e:
            logging.error(f"Erreur conventions pour {url}: {e}")
            try:
                with open(f"error_page_{result['ID']}.html", "w", encoding="utf-8") as f:
                    f.write(driver.page_source)
            except:
                pass

        # Date de diplôme
        try:
            date_diplome_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-testid='diplomas-section'] span.cv-date"))
            )
            if date_diplome_elements:
                result['Date de diplôme'] = date_diplome_elements[0].text.strip()
        except Exception as e:
            logging.warning(f"Date de diplôme non trouvée pour {url}: {e}")

        # Horaires
        try:
            horaires_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul li.mui-182gbo5"))
            )
            horaires_med = {}
            for element in horaires_elements:
                jour = element.find_element(By.CSS_SELECTOR, "span.mui-1wwj1lu").text.strip()
                horaires_plage = element.find_element(By.CSS_SELECTOR, "span.mui-9rvw1i").text.strip()
                horaires_med[jour] = horaires_plage
            
            horaires_str = ", ".join([f"{jour}: {plage}" for jour, plage in horaires_med.items()])
            result['Horaires'] = horaires_str
        except Exception as e:
            logging.warning(f"Horaires non trouvés pour {url}: {e}")

    except Exception as e:
        logging.error(f"Erreur générale pour {url}: {e}")
    
    finally:
        driver.quit()
        return result

def scrape_all_urls(medecin_urls):
    data = []
    
    # Réduire le nombre de workers pour éviter les problèmes de concurrence
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(scrape_url, url) for url in medecin_urls]
        for future in concurrent.futures.as_completed(futures):
            data.append(future.result())
            time.sleep(1)  # Petit délai entre chaque requête
    
    return data

# Exemple d'utilisation
if __name__ == "__main__":

    all_data = scrape_all_urls(medecin_urls)
    df_indiv = pd.DataFrame(all_data)



df_indiv.to_csv("BASE_SECOURS/SELLENIUM_ile_de_france.csv",sep=';', index=False, encoding="utf-8")


In [ ]:
data = pd.merge(df_indiv, df, on='URL', how='outer')
df_indiv.to_csv("DATA/ile_de_france.csv", index=False, encoding="utf-8")

10